In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["subcategory"] == "Cost per MWh delivered")
    & (df["esc"].isin(["hvdc-to-elec", "pipeline-h2-to-elec"]))
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "annually": "Annual",
    "quaterly": "Quarterly",
    "monthly": "Month",
    "biweekly": "Biweekly",
    "weekly": "Weekly",
    "daily": "Daily",
    "unbuffered": "Baseload",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

# Construct labels with nicer ESC names and CSP indicator
df["esc"] = df["esc"].replace(
    {"hvdc-to-elec": "HVDC", "pipeline-h2-to-elec": "H2 Pipeline"}
)
df.loc[df["csp"], "esc"] += " (with CSP)"
df.loc[~df["csp"], "esc"] += " (no CSP)"


df["xposition"] = (
    df["flexibility"]
    .replace({v: idx for (idx, v) in enumerate(flexibilities.values())})
    .astype(int)
)

df.loc[df["exporter"] == "TN", "xposition"] += len(flexibilities) + 1

df = df.set_index(["exporter", "esc", "xposition"])[
    ["flexibility", "value"]
].sort_values("xposition")

fig, ax = plt.subplots(figsize=(12, 4))

handles, labels = [], []
for exp, esc, csp in itertools.product(
    ["MA", "TN"], ["HVDC", "H2 Pipeline"], ["(with CSP)", "(no CSP)"]
):
    label = f"{esc} {csp}"

    color = {"HVDC": "black", "H2 Pipeline": "green"}
    color = color[esc]

    mfcolor = {"HVDC": "black", "H2 Pipeline": "white"}
    mfcolor = mfcolor[esc]

    marker = {"HVDC": "*", "H2 Pipeline": "o"}
    marker = marker[esc]

    linestyle = {"(with CSP)": "solid", "(no CSP)": "dotted"}
    linestyle = linestyle[csp]

    handle = df.xs((exp, label)).plot(
        ax=ax,
        marker=marker,
        color=color,
        markerfacecolor=mfcolor,
        ls=linestyle,
        label=label,
    )

    labels.append(label)

ax.text(
    3,
    200 + 25 / 2,
    "Morocco (MA)",
    ha="center",
    va="center",
    bbox=dict(boxstyle="square, pad=0.2", fc="white", ec="none"),
)
ax.text(
    11,
    200 + 25 / 2,
    "Tunisia (TN)",
    ha="center",
    va="center",
    bbox=dict(boxstyle="square, pad=0.2", fc="white", ec="none"),
)
ax.plot([len(flexibilities)] * 2, [0, 250], ls="dashed", c="grey")

# ax.legend(ax.get_legend_handles_labels()[0][:4][::-1], labels[:4][::-1],
#          loc="upper right",
#          bbox_to_anchor=(1,-0.1),
#         )

# Legend right to plot, all four labels stacked
# ax.legend(
#     ax.get_legend_handles_labels()[0][:4][::-1],
#     labels[:4][::-1],
#     ncols=1,
#     loc="lower left",
#     framealpha=0.9,
#     bbox_to_anchor=(1.05, 0),
# )

# Legend center of plot, not stacked
ax.legend(
    ax.get_legend_handles_labels()[0][:4][::-1],
    labels[:4][::-1],
    ncols=2,
    loc="lower center",
    framealpha=0.9,
    bbox_to_anchor=(0.5, 0),
)

# ax.legend(ax.get_legend_handles_labels()[0][:4][::-1], labels[:4][::-1],
#          ncols=4,
#          loc="upper center",
#          framealpha=.9,
#          bbox_to_anchor=(.5,-.2),
# )

ax.set_axisbelow(True)
ax.set_yticks(np.arange(25, 275, 50), minor=True)
ax.set_xticks(df.xs("HVDC (no CSP)", level=1).reset_index()["xposition"].values)
ax.set_xticklabels(df.xs("HVDC (no CSP)", level=1)["flexibility"].astype(str).values)
ax.yaxis.grid(which="minor", ls="--")
ax.set_ylabel("Electricity cost [EUR/MWh]")
ax.set_xlabel("Demand matching")
ax.set_xlim(df.reset_index()["xposition"].agg(["min", "max"]).values + [-0.2, 0.2])
ax.set_ylim(0, 225)  # Magic number 200: Highest value from TN ESC H2 pipeline

ax.tick_params(left=True, right=True, labelleft=True, labelright=True)

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")